# Migration of SQLAlchemy Version 1.4 to version 2 using Gemini dev models

we will use Gemini model through exposed API  using the Generativeai Python SDK

## Prerequisites

You can run this tutorial in Google Colab, which doesn't require additional environment configuration.


## workflow

You need just to import the python repository file inside sample_data and then after runing all cells it will generate migrated version inside migration generated folder

## Install the SDK

The Python SDK for the Gemini API is contained in the [`google-generativeai`](https://pypi.org/project/google-generativeai/) package. Install the dependency using pip:

In [1]:
!pip install -q -U google-generativeai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.0/165.0 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 725.4/725.4 kB 33.8 MB/s eta 0:00:00


## Set up your API key

To use the Gemini API, you'll need an API key. If you don't already have one, create a key in Google AI Studio.

<a class="button" href="https://aistudio.google.com/app/apikey" target="_blank" rel="noopener noreferrer">Get an API key</a>

In Colab, add the key to the secrets manager under the "🔑" in the left panel. Give it the name `GOOGLE_API_KEY`. Then pass the key to the SDK:

In [2]:
# Import the Python SDK
import google.generativeai as genai

# Used to securely store your API key
from google.colab import userdata

GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

# read python code

This function will read the python code into string content to be passed to gemini LLM in it's prompt

In [5]:
def read_python_code(file_path):
    with open(file_path, 'r') as file:
        code = file.read()
    return code

# Usage example
file_path = '/content/sample_data/user_repository.py'
code_string = read_python_code(file_path)
#print(code_string)


# Initialize the Generative Model

Before you can make any API calls, you need to initialize the Generative Model.

we choose the gemini v1 pro

Here some available models :

# Model Variants Overview

This documentation describes various model variants optimized for different tasks. Each model is designed to handle specific input types and produce optimized output.

## 1. Gemini 1.5 Flash

- **Model Variant:** gemini-1.5-flash
- **Input(s):** Audio, images, videos, and text
- **Output:** Text
- **Optimized for:** Fast and versatile performance across a diverse variety of tasks.

---

## 2. Gemini 1.5 Pro

- **Model Variant:** gemini-1.5-pro
- **Input(s):** Audio, images, videos, and text
- **Output:** Text
- **Optimized for:** Complex reasoning tasks such as code and text generation, text editing, problem solving, data extraction, and generation.

---

## 3. Gemini 1.0 Pro

- **Model Variant:** gemini-1.0-pro
- **Input(s):** Text
- **Output:** Text
- **Optimized for:** Natural language tasks, multi-turn text and code chat, and code generation.

---

## 4. Text Embedding

- **Model Variant:** text-embedding-004
- **Input(s):** Text
- **Output:** Text embeddings
- **Optimized for:** Measuring the relatedness of text strings.

---

## 5. AQA (Answer Quality Assurance)

- **Model Variant:** aqa
- **Input(s):** Text
- **Output:** Text
- **Optimized for:** Providing source-grounded answers to questions.


Since gemini-1.5-pro has emitted a 500 tornado error due to a request timeout, we will use gemini-1.0-pro and gemini-1.5-flash for comparison in the report.

Load the model

In [6]:
model = genai.GenerativeModel('gemini-1.0-pro')

# Generate text

Prepare system prompt

In [7]:
migration_instructions = '''The Python code bellow uses the library sqlalchemy with version 1. Migrate it so that it works with version 2 of sqlalchemy.
Make the code compatible with python’s asyncio. Use python’s typing module to add type hints to the code. Your answer must only contain code. Do not explain it. Do not add markdown backticks for code. Do not add extra functionality to the code. Do not remove code that is not being changed. If there’s no need to change the code, answer only with the code itself. The first line of code must have a comment "### START CODE ###". The last line of code must have a comment "### END CODE ###".

'''


In [ ]:
new_zero_shot_migration_instructions = '''The Python code below uses the library SQLAlchemy version 1.4. It's a user repository file. Migrate it so that it works with version 2 of SQLAlchemy. Make the code compatible with Python’s asyncio. Use Python’s typing module to add type hints to the code. Your answer must only contain code. Do not explain it. Do not add markdown backticks for code. Do not add extra functionality to the code. Do not remove code that is not being changed. If there’s no need to change the code, answer only with the code itself. Please try to implement the migrated methods; do not leave necessary methods unimplemented, and do not forget all the necessary imports. The first line of code must have a comment "### START CODE ###". The last line of code must have a comment "### END CODE ###". Here is the code to migrate:
### START CODE ###
'''


In [8]:
one_shot_example = """Before migration:

from typing import List from sqlalchemy.orm import sessionmaker, declarative_base from sqlalchemy import Column, Integer, String, create_engine from sqlalchemy.future import select

DATABASE_URL = "sqlite:///./test.db"

engine = create_engine(DATABASE_URL, echo=True) SessionLocal = sessionmaker(autocommit=False, autoflush=False, bind=engine) Base = declarative_base()

class User(Base): tablename = "users"

mathematica

id = Column(Integer, primary_key=True, index=True)
name = Column(String, index=True)

def get_user_by_id(session: SessionLocal, user_id: int) -> User: with session() as db_session: result = db_session.execute(select(User).filter(User.id == user_id)) return result.scalars().one_or_none()

def get_all_users(session: SessionLocal) -> List[User]: with session() as db_session: result = db_session.execute(select(User)) return result.scalars().all()

After migration:

from typing import List from sqlalchemy.ext.asyncio import AsyncSession, create_async_engine from sqlalchemy.orm import sessionmaker, declarative_base, mapped_column from sqlalchemy import Integer, String from sqlalchemy.future import select

DATABASE_URL = "sqlite+aiosqlite:///./test.db"

engine = create_async_engine(DATABASE_URL, echo=True) SessionLocal = sessionmaker(autocommit=False, autoflush=False, bind=engine, class_=AsyncSession) Base = declarative_base()

class User(Base): tablename = "users"

mathematica

id = mapped_column(Integer, primary_key=True, index=True)
name = mapped_column(String, index=True)

async def get_user_by_id(session: AsyncSession, user_id: int) -> User: async with session() as db_session: result = await db_session.execute(select(User).filter(User.id == user_id)) return result.scalar_one_or_none()

async def get_all_users(session: AsyncSession) -> List[User]: async with session() as db_session: result = await db_session.execute(select(User)) return result.scalars().all()"""

In [14]:
chain_of_thought_migration_guide = """
Use the steps below as a guide for the migration. You don’t
need to follow them exactly as described, but they should be
able to help with the migration:
1. Update the used database engine, if any, so that you’re using
‘create_async_engine‘ instead of ‘create_engine‘.
2. If any tables and their columns are declared, update their
declarations so that they use ‘mapped_columns‘ instead of
‘schema.Column‘ and ensure they are correctly typed with the
Mapped annotation, making sure to import the correct types
from the library.
3. Ensure that all queries, if any, are updated to use the new 2.0
style of querying, such as using ‘select()‘ instead of ‘query()‘.
4. Update functions that use ‘sessionmaker‘ to use ‘session‘ instead.
5. Update the code to use async functions and await calls where
necessary.
6. Implement type hinting for all functions and variables and update old type hinting to ensure they are correct.
7. Ensure there are no missing import statements.
8. Remove any unused imports or variable declarations.
9. Make sure the code works.
"""


In [9]:
prompt = f"{migration_instructions}\n here  is an example of code before and after migration follow it {one_shot_example} , Here is the code to migrate: ### START CODE ###  {code_string}"

In [ ]:
prompt = f"{migration_instructions}\n {chain_of_thought_migration_guide} , Here is the code to migrate: ### START CODE ###  : {code_string}"

Length of the message

In [10]:
len(f"{prompt}")

8594

In [11]:
prompt

'The Python code bellow uses the library sqlalchemy with version 1. Migrate it so that it works with version 2 of sqlalchemy.\nMake the code compatible with python’s asyncio. Use python’s typing module to add type hints to the code. Your answer must only contain code. Do not explain it. Do not add markdown backticks for code. Do not add extra functionality to the code. Do not remove code that is not being changed. If there’s no need to change the code, answer only with the code itself. The first line of code must have a comment "### START CODE ###". The last line of code must have a comment "### END CODE ###".\n\n\n here  is an example of code before and after migration follow it Before migration:\n\nfrom typing import List from sqlalchemy.orm import sessionmaker, declarative_base from sqlalchemy import Column, Integer, String, create_engine from sqlalchemy.future import select\n\nDATABASE_URL = "sqlite:///./test.db"\n\nengine = create_engine(DATABASE_URL, echo=True) SessionLocal = ses

Generate response

In [12]:
response = model.generate_content(prompt)
print(response.text)

### START CODE ###
import os
from functools import lru_cache
from typing import Optional, Iterator, List

from pydantic import BaseModel
from sqlalchemy.ext.asyncio import create_async_engine, AsyncSession, sessionmaker
from sqlalchemy.orm import sessionmaker, Session, declarative_base, mapped_column
from sqlalchemy import Integer, String
from sqlalchemy.future import select

SQL_BASE = declarative_base()


@lru_cache(maxsize=None)
def get_engine(db_string: str):
    """
        Create and cache a SQLAlchemy engine.
        """
    return create_async_engine(db_string, pool_pre_ping=True)


class UserInDB(SQL_BASE):
    """
      SQLAlchemy model representing a user in the database.
      """
    __tablename__ = 'user_table'

    id = mapped_column(Integer, primary_key=True, autoincrement=True)
    email = mapped_column(String(length=128), unique=True, nullable=False)
    password = mapped_column(String(length=128), nullable=False)
    name = mapped_column(String(length=128), nullable=

# write string response back to python file

In [13]:
import os
def write_python_code(file_path: str, code: str) -> None:
    # Ensure the directory exists
    os.makedirs(os.path.dirname(file_path), exist_ok=True)
    with open(file_path, 'w') as file:
        file.write(code)
# Usage example
migrated_file_path = '/content/output/user_repository.py'

write_python_code(migrated_file_path, response.text)


#Change some generation Hyperparameters

In [ ]:
response = model.generate_content(
    "Tell me a story about a magic backpack.",
    generation_config=genai.types.GenerationConfig(
        # Only one candidate for now.
        candidate_count=1,
        stop_sequences=["x"],
        max_output_tokens=20,
        temperature=1.0,
    ),
)

print(response.text)


Flora, a girl with eyes the color of a stormy sea and a head full of daydreams,
